## Customer Dataset Cleaning

In [1]:
%pip install fuzzywuzzy
!python -m pip install --upgrade pip
%pip install python-Levenshtein
%pip install pandas numpy matplotlib seaborn scikit-learn jupyter

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Using cached matplotlib-3.10.7-cp312-cp312-win_amd64.whl.metadata (11 kB)
  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached scikit_learn-1.7.2-cp312-cp312-win_amd64.whl.metadata (11 kB)
  Using cached jupyter-1.1.1-py2.py3-none-any.whl.metadata (2.0 kB)
  Using cached contourpy-1.3.3-cp312-cp312-win_amd64.whl.metadata (5.5 kB)
  Using cached jupyter_console-6.6.3-py3-none-any.whl.metadata (5.8 kB)
  Using cached ipywidgets-8.1.7-py3-none-any.whl.metadata (2.4 kB)
Using cached matplotlib-3.10.7-cp312-cp312-win_amd64.whl (8.1 MB)
Using cached seaborn-0.13.2-py3-none-any.whl (294 kB)
Using cached scikit_learn-1.7.2-cp312-cp312-win_amd64.whl (8.7 MB)
Using cached jupyter-1.1.1-py2.py3-none-any.whl (2.7 kB)
Using cached contourpy-1.3.3-cp312-cp312-win_amd64.whl (226 kB)
Using cached ipywidgets-8.1.7-py3-none-any.whl (139 kB)
Usin

In [2]:
# Activate kernal:
# .\venv_fyp\Scripts\activate

# Step 1: Import libraries
import pandas as pd
import numpy as np
import os
from datetime import datetime
from fuzzywuzzy import process


In [3]:
# Step 2: Load dataset
# Replace 'customer_dataset.csv' with your actual file name or path
file_path = "C:/Users/user/OneDrive/Desktop/Onedrive_YuyanDipsy/OneDrive/UM Y4S1/WIA3002 FYP 1 & 2/FYP2/Data/Soapan Santun/2021 - 2025 Customer.csv"

original_dataset_name = "2021 - 2025 Customer.csv"

# Read dataset
df = pd.read_csv(file_path)

# Show first few rows
df.head()

,CustomerID,Date of Birth,Gender,Income Level,City,State,Country
0,CUST0001,NaN,female,NaN,Sungai Besi,Kuala Lumpur,Malaysia
1,CUST0001,NaN,female,NaN,NaN,NaN,NaN
2,CUST0002,NaN,female,NaN,Mutiara Damansara,Selangor,Malaysia
3,CUST0002,NaN,female,NaN,NaN,NaN,NaN
4,CUST0003,NaN,female,NaN,Shah Alam,Selangor,Malaysia


### Data Profiling

In [ ]:
## Create later

### Initial checking: (Before perform data cleaning) Check optional and mandatory columns 

In [ ]:
# Optional columns
def check_optional_columns(df, threshold=0.5):
    """
    Check optional columns for fill percentage and drop columns that are mostly empty.
    Returns the modified DataFrame and a friendly message.
    """
    
    optional_columns = ["Date of Birth", "Gender", "Income Level"]

    # Normalize column names
    df.columns = df.columns.str.strip().str.lower()
    optional_columns = [col.lower() for col in optional_columns]

    dropped_columns = []

    for col in optional_columns:
        if col in df.columns:
            fill_ratio = df[col].count() / len(df) # 
            if fill_ratio < threshold:
                dropped_columns.append(col)
                df.drop(columns=[col], inplace=True)  # Drop the column immediately
                # df[col].count(): This counts the number of non-missing (non-null/non-NaN) values in the current column (col).
                # len(df): This gives the total number of rows in the DataFrame.
                # fill_ratio: The division calculates the proportion of filled (non-missing) values in that column. A ratio of 1.0 means the column is entirely filled; a ratio of 0.1 means 90% of the values are missing.


    # Generate user-friendly message
    if dropped_columns:
        dropped_str = ", ".join(dropped_columns)
        message = (
            f"We noticed that very few entries were provided for {dropped_str}. "
            "These columns have been removed. "
            "Segmentation will still be performed using geographic (City, State, Country) "
            "and behavioral data (e.g., orders, purchase items, total spend)."
        )
    else:
        message = "All optional columns have enough data and are kept for analysis."
    
    return df, message

# Mandatory columns 
def check_mandatory_columns(df, threshold=0.2):
    """
    Check mandatory columns for missing values (column-wise).
    Does not drop columns — only warns user if any column is too incomplete.
    Returns the DataFrame and a message summarizing issues.
    """

    mandatory_columns = ["CustomerID", "City", "State", "Country"]

    # Normalize column names
    df.columns = df.columns.str.strip().str.lower()
    mandatory_columns = [col.lower() for col in mandatory_columns]

    missing_report = []
    warning_columns = []

    for col in mandatory_columns:
        if col in df.columns:
            fill_ratio = df[col].count() / len(df)
            missing_percent = (1 - fill_ratio) * 100

            missing_report.append(f"{col}: {missing_percent:.1f}% missing")

            # Warn if missing exceeds threshold
            if fill_ratio < (1 - threshold):
                warning_columns.append(col)
        else:
            # Handle case where column completely missing
            missing_report.append(f"{col}: column not found (100% missing)")
            warning_columns.append(col)

    # Generate friendly message
    if warning_columns:
        warning_str = ", ".join(warning_columns)
        message = (
            f"Some key fields have a high number of missing values: {warning_str}. "
            "The system will still continue cleaning and processing, "
            "but missing values will be handled automatically by our system. "
            "Please ensure your source data is as complete as possible for more accurate segmentation results.\n\n"
            "Missing Data Summary:\n" + "\n".join(missing_report)
        )
    else:
        message = (
            "All mandatory columns have sufficient data and are ready for cleaning.\n\n"
            "Missing Data Summary:\n" + "\n".join(missing_report)
        )

    return df, message


In [5]:
# --- Apply checks and save cleaned dataset ---
# Step 1: Optional columns
df, optional_check_message = check_optional_columns(df)

# Step 2: Mandatory columns (just check, but keep all columns)
df, mandatory_check_message = check_mandatory_columns(df)

# Step 3: Save the updated dataset for cleaning

# Split the name and extension
base_name, ext = os.path.splitext(original_dataset_name)

# Create new cleaned file name
cleaned_file = f"{base_name}_cleaned{ext}"

# Save the cleaned dataset
df.to_csv(cleaned_file, index=False)

# --- Display results ---
print("Optional Columns Check:")
print(optional_check_message)

print("\nMandatory Columns Check:")
print(mandatory_check_message)

print(f"\nCleaned dataset saved as '{cleaned_file}'")


Optional Columns Check:
We noticed that very few entries were provided for date of birth, income level. These columns have been removed. Segmentation will still be performed using geographic (City, State, Country) and behavioral data (e.g., orders, purchase items, total spend).

Mandatory Columns Check:
Some key fields have a high number of missing values: city, state, country. The system will still continue cleaning and processing, but missing values will be handled automatically by our system. Please ensure your source data is as complete as possible for more accurate segmentation results.

Missing Data Summary:
customerid: 0.0% missing
city: 44.8% missing
state: 46.9% missing
country: 44.7% missing

Cleaned dataset saved as '2021 - 2025 Customer_cleaned.csv'


In [6]:
# After perform initial checking on optional and mandatory columns
df.head()

,customerid,gender,city,state,country
0,CUST0001,female,Sungai Besi,Kuala Lumpur,Malaysia
1,CUST0001,female,NaN,NaN,NaN
2,CUST0002,female,Mutiara Damansara,Selangor,Malaysia
3,CUST0002,female,NaN,NaN,NaN
4,CUST0003,female,Shah Alam,Selangor,Malaysia


### Perform Data Cleaning

In [ ]:
def clean_customer_dataset(df, order_df=None):
    # Step 1: Normalize column names
    df.columns = df.columns.str.strip().str.lower()

    # Step 2: Remove exact duplicate rows
    df.drop_duplicates(inplace=True)

    # Step 3: Clean CustomerID
    if "customerid" in df.columns:
        df["customerid"] = df["customerid"].astype(str).str.strip().str.upper()

    # Step 4: Handle optional columns
    optional_cols = ["date of birth", "gender", "income level"]
    for col in optional_cols:
        if col not in df.columns:
            continue

        if col == "date of birth":
            df[col] = pd.to_datetime(df[col], errors="coerce", infer_datetime_format=True)
            if df[col].isnull().mean() > 0.5:
                df.drop(columns=[col], inplace=True)

        if col == "gender":
            df[col] = df[col].astype(str).str.strip().str.lower()
            gender_map = {"m": "Male", "male": "Male", "f": "Female", "female": "Female"}
            df[col] = df[col].map(gender_map).fillna("Other")

        if col == "income level":
            df[col] = df[col].astype(str).str.strip().str.lower()
            income_map = {
                "low": "Low", "lower": "Low",
                "medium": "Medium", "mid": "Medium", "med": "Medium", "middle": "Medium",
                "high": "High", "upper": "High"
            }
            df[col] = df[col].replace(income_map)
            if df[col].isnull().mean() > 0.5:
                df.drop(columns=[col], inplace=True)

    # Step 5: Clean City, State, Country
    for col in ["city", "state", "country"]:
        if col not in df.columns:
            continue
        df[col] = df[col].astype(str).str.strip().str.title()

    # Step 6: Cross-check with orders (if provided)
    if order_df is not None and "customerid" in order_df.columns:
        valid_ids = set(order_df["customerid"].astype(str).str.strip().str.upper())
        df = df[df["customerid"].isin(valid_ids)]

    # ✅ Step 7: Handle duplicate customerid at the end
    if "customerid" in df.columns:
        def combine_duplicates(group):
            if len(group) == 1:
                return group.iloc[0]
            if group.drop(columns="customerid").nunique().sum() > (group.shape[1] - 1):
                combined = {}
                for col in group.columns:
                    if col == "customerid":
                        combined[col] = group[col].iloc[0]
                    else:
                        if pd.api.types.is_numeric_dtype(group[col]):
                            combined[col] = group[col].mean()
                        else:
                            combined[col] = " / ".join(group[col].dropna().unique().astype(str))
                return pd.Series(combined)
            else:
                return group.iloc[0]

        df = df.groupby("customerid", group_keys=False).apply(combine_duplicates).reset_index(drop=True)

    # Step 8: Return cleaned dataset
    return df
